In [ ]:
import folium
import pandas as pd
import json
import matplotlib as plt
import matplotlib

In [ ]:
vmin = 0
vmax = 9

normalizer = plt.colors.Normalize(vmin = vmin, vmax = vmax)
cmap = plt.cm.get_cmap('YlGn', 5)    # PiYG
color_map = plt.cm.ScalarMappable(norm=normalizer, cmap=cmap)

In [ ]:
cmap = plt.cm.get_cmap('YlGn', 10)    # PiYG
cmap

In [ ]:
x = 7.9
c = color_map.to_rgba(x)
c = plt.colors.to_hex(c)
c

In [ ]:
def get_color_mapper(vmin, vmax, cmap="seismic", ncolors=10):
    normalizer = plt.colors.Normalize(vmin = vmin, vmax = vmax)
    cmap = plt.cm.get_cmap(cmap, ncolors)    # PiYG
    color_map = plt.cm.ScalarMappable(norm=normalizer, cmap=cmap)
    
    def color_mapper(x):
        c = color_map.to_rgba(x)
        c = plt.colors.to_hex(c)
        return c
    return color_mapper

## Markers

In [ ]:
my_map = folium.Map(
    location=(13.0, 16.0),
    max_bounds=True,
    zoom_start=2,
)

In [ ]:
cities = pd.read_csv("cities.txt")
cities.head(3)

In [ ]:
city = cities.loc[0]

In [ ]:
for _, city in cities.iterrows():
    folium.Marker(
        location=[city["latitude"], city["longitude"]],
        popup=city["name"],
        tooltip=city["name"]
    ).add_to(my_map)

In [ ]:
p_world_countries = f'world-countries.json'

with open(p_world_countries, 'r') as f:
        world_countries = json.load(f)
country_names = [country["properties"]["name"] for country in world_countries["features"]]
len(country_names), len(world_countries["features"])

### World

In [ ]:
p_world_countries = f'countries_high_res.json'

with open(p_world_countries, 'rb') as f:
        world_countries = json.load(f)

country_names = [country["properties"]["name"] for country in world_countries["features"]]
        
len(world_countries["features"])

In [ ]:
country_names = sorted(country_names)
country_data = list(zip(country_names, range(len(country_names))))
country_data = pd.DataFrame(country_data, columns=["name", "value"])
country_data = country_data[:50]

In [ ]:
country_costs = []
country_security = []
country_climbing = []
country_interest = []

In [ ]:
prices = pd.read_csv("prices.csv").sort_values(by="country")
prices = prices.groupby('country', as_index=False)['price'].mean().sort_values(by="country")
prices = prices.reset_index(drop=True)
country_prices = dict(zip(prices.country, prices.price))

In [ ]:
for f in world_countries["features"]:
    name = f["properties"]["name"]
    if name in country_prices:
        p = country_prices[name]
        f["properties"]["price"] = float(str("{:.1f}".format(p)))
    else:
        f["properties"]["price"] = None
    

In [ ]:
cmap_name = 'YlGn'
ncolors = 10
cmap = plt.cm.get_cmap(cmap_name, ncolors)    # PiYG

color_mapper = get_color_mapper(vmin=prices["price"].min(), vmax=prices["price"].max(), cmap=cmap_name, ncolors=ncolors)
cmap

In [ ]:
def price_function(x):
    """
        fillcolor: color if area
        color: color of border liens
        weight: size of border lines
    """
    country_name = x["properties"]["name"]
    style_dict = {'fillColor': "gray", 'color': 'gray', "opacity": 0, "weight": 0, 'fillOpacity': 1.0}
    if country_name in country_prices:
        c = country_prices[country_name]
        c = color_mapper(c)
        style_dict = {'fillColor': c, 'weight':1.0, "color": c, "opacity": 1.0, "fillOpacity": 0.7}
    return style_dict

In [ ]:
from branca.element import Figure
fig=Figure(width=1000,height=600)
m = folium.Map(zoom_start=2)
fig.add_child(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)

folium.GeoJson(world_countries,
               name = 'prices',
               style_function = price_function,
               
               tooltip = folium.GeoJsonTooltip(fields=('name', 'price'), aliases=('name', 'price'), labels=True),
               zoom_on_click=True,
               show = True).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
from branca.element import Figure
fig=Figure(width=1000,height=600)
m = folium.Map(zoom_start=2)
fig.add_child(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)

folium.Choropleth(
    geo_data=world_countries,
    name="prices",
    data=prices,
    columns=["country", "price"],
    key_on="feature.properties.name",
    fill_color="YlGn",
    fill_opacity=1.0,
    line_opacity=0.2,
    legend_name="Prices",
).add_to(m)

folium.LayerControl().add_to(m)
m